<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
!تشخیص تقلب
</font>
</h1>

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معرفی مجموعه داده
</font>
</h2>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>

</font>
</p>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
در این قسمت کتاب‌خانه‌ها و ابزار مورد نیاز خود را <code>import</code> کنید و فایل داده‌ها را که در پوشه‌ی <code>Data</code> و که با نام‌های <code>train.csv</code> و <code>test.csv</code> ذخیره‌شده‌اند را بخوانید و وارد محیط کار خود کنید.
</font>
</p>

In [151]:
# Classification imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC



<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
پیش‌پردازش و مهندسی ویژگی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
        در این سوال شما می‌توانید از هر تکنیک پیش‌پردازش/مهندسی ویژگی دلخواهتان، استفاده کنید.
    <br>
    تکنیک‌هایی که استفاده می‌کنید به شکل مستقیم مورد ارزیابی توسط سامانه داوری قرار <b>نمی‌گیرند.</b> بلکه همه آن‌ها در دقت مدل شما تاثیر خواهند گذاشت؛ بنابراین هر چه پیش‌پردازش/مهندسی ویژگی بهتری انجام دهید تا دقت مدل بهبود پیدا کند، امتیاز بیشتری از این سوال کسب خواهید کرد.
    در این قسمت شما می‌توانید بخشی از داده‌ی موجود را برای اعتبارسنجی در نظر بگیرید.
</font>
</p>

In [152]:
train = pd.read_csv("./Data/train.csv")
x_test = pd.read_csv("./Data/test.csv")

In [153]:
train.drop(columns=["user_id"],inplace=True)
userid = x_test["user_id"]
x_test.drop(columns=["user_id"],inplace=True)
train.drop(columns=["timestamp"],inplace=True)
x_test.drop(columns=["timestamp"],inplace=True)

In [154]:
train["payment_method"].value_counts()


payment_method
credit_card    4350
paypal         2908
debit_card     2342
Name: count, dtype: int64

In [155]:
train["product_category"].value_counts()


product_category
subscription       2643
digital_service    2160
travel             1636
apparel            1584
electronics        1577
Name: count, dtype: int64

In [156]:
encoder = LabelEncoder()
train["product_category"] = encoder.fit_transform(train["product_category"])


x_test["product_category"] = encoder.transform(x_test["product_category"])


In [157]:
encoder = LabelEncoder()
train["payment_method"] = encoder.fit_transform(train["payment_method"])


x_test["payment_method"] = encoder.transform(x_test["payment_method"])

In [158]:
train

,processing_batch_id,transaction_amount,user_age_days,ip_risk_score,transaction_speed_seconds,product_category,payment_method,is_fraud
0,2113,4.27,392,30.01,17.73,3,0,0
1,2254,37.62,507,25.83,22.36,4,1,0
2,2373,531.63,64,85.22,9.35,1,0,1
3,2158,314.96,965,55.09,22.81,3,2,0
4,2381,2300.59,45,94.31,4.67,1,0,1
...,...,...,...,...,...,...,...,...
9595,2353,18.18,579,68.71,18.31,3,0,0
9596,2328,14.98,15,19.19,20.50,1,0,0
9597,2129,27.36,931,54.98,16.88,1,1,0
9598,2331,126.54,196,61.25,17.08,0,2,0


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
آموزش مدل
</font>
</h2>



In [159]:
X = train.drop(columns=["is_fraud"])
y = train["is_fraud"]

X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.5,random_state=42)

In [160]:
scaller = StandardScaler()


X_train = scaller.fit_transform(X_train)
X_val = scaller.transform(X_val)
x_test = scaller.transform(x_test)

In [161]:

# --- Helper function for metrics ---
def evaluate_model(name, y_true, y_pred, y_prob=None):
    print(f"\n{name} Results:")
    print("-" * 40)
    print("Accuracy :", accuracy_score(y_true, y_pred))
    # print("Precision:", precision_score(y_true, y_pred))
    # print("Recall   :", recall_score(y_true, y_pred))
    print("F1 Score :", f1_score(y_true, y_pred))
  
    print("-" * 40)

# --- XGBoost Classifier ---
xgb_model = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    eval_metric='logloss'
)
xgb_model.fit(X_train, y_train)

xgb_preds = xgb_model.predict(X_val)
xgb_preds_train = xgb_model.predict(X_train)

evaluate_model("XGBoost", y_val, xgb_preds)
evaluate_model("XGBoost train", y_train, xgb_preds_train)

# --- Support Vector Machine ---
# svm_model = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)
# svm_model.fit(X_train, y_train)

# svm_preds = svm_model.predict(X_val)
# svm_preds_train = svm_model.predict(X_train)

# evaluate_model("SVM", y_val, svm_preds)
# evaluate_model("SVM train", y_train, svm_preds_train)

# --- Random Forest Classifier ---
# rf_model = RandomForestClassifier(n_estimators=200, random_state=42)
# rf_model.fit(X_train, y_train)

# rf_preds = rf_model.predict(X_val)
# rf_preds_train = rf_model.predict(X_train)

# evaluate_model("Random Forest", y_val, rf_preds)
# evaluate_model("Random Forest train", y_train, rf_preds_train)


XGBoost Results:
----------------------------------------
Accuracy : 0.9952083333333334
F1 Score : 0.9909198578760363
----------------------------------------

XGBoost train Results:
----------------------------------------
Accuracy : 1.0
F1 Score : 1.0
----------------------------------------


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معیار ارزیابی
</font>
</h2>



In [162]:
from sklearn.metrics import accuracy_score

# TODO:

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
 پیش‌بینی برای داده تست و خروجی
</font>
</h2>


In [163]:
# TODO:

submission = pd.DataFrame({"user_id":userid,"is_fraud": xgb_model.predict(x_test)})
submission

,user_id,is_fraud
0,100811,1
1,104345,1
2,100383,0
3,111014,0
4,110828,0
...,...,...
2395,110834,0
2396,102130,1
2397,109608,0
2398,109950,0


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) در غیر این صورت، در پایان مسابقه نمره شما به صفر تغییر خواهد کرد.
    <br>
    همچنین اگر از کولب برای اجرای این فایل نوت‌بوک استفاده می‌کنید، قبل از ارسال فایل <code>result.zip</code>، آخرین نسخه‌ی نوت‌بوک خود را دانلود کرده و داخل فایل ارسالی قرار دهید.
</font>

In [164]:
import zipfile
import joblib
import os

if not os.path.exists(os.path.join(os.getcwd(), 'fraudy.ipynb')):
    %notebook -e fraudy.ipynb

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

submission.to_csv('submission.csv', index=False)
file_names = ['fraudy.ipynb', 'submission.csv']
compress(file_names)

File Paths:
['fraudy.ipynb', 'submission.csv']
